In [16]:
# 1. Importer les bibliothèques nécessaires
import pandas as pd
import json
import folium
from shapely.geometry import Polygon
import os

os.makedirs('../generated/sites', exist_ok=True)
source_dir = '../generated/'
faits = pd.read_csv(f"{source_dir}faits_matieres.csv", sep=',', encoding='utf-8')
dim_type_matiere = pd.read_csv(f"{source_dir}dim_type_matiere.csv", sep=',', encoding='utf-8')
dim_producteur = pd.read_csv(f"{source_dir}dim_producteur.csv", sep=',', encoding='utf-8')
dim_temps = pd.read_csv(f"{source_dir}dim_temps.csv", sep=',', encoding='utf-8')
dim_site_valorisation = pd.read_csv(f"{source_dir}dim_site_valorisation.csv", sep=',', encoding='utf-8')

geojson_path = "../generated/normalized/collecte.json"

In [ ]:
# 2. Charger les données tabulaires et géospatiales

# with open(geojson_path, "r") as f:
#     geo_data = json.load(f)

# features = geo_data['features']
# geo_df = pd.json_normalize(features)
# # Ajouter la colonne PROVINCE='Quebec' si MUNICIPALITE == 'Montreal' dans geo_df
# if 'properties.MUNICIPALITE' in geo_df.columns:
#     geo_df.loc[geo_df['properties.MUNICIPALITE'] == 'Montreal', 'properties.PROVINCE'] = 'Québec'

# geo_df = geo_df[['properties.PROVINCE','properties.MUNICIPALITE','properties.SECTEUR', 'properties.JOUR', 'geometry.type', 'geometry.coordinates']]
# geo_df.head()

# # 3. Fusionner les données sur le secteur
# faits_geo = faits.merge(geo_df, left_on='geo', right_on='properties.PROVINCE', how='left')


# 4. Visualiser la carte interactive
# m = folium.Map(location=[45.5017, -73.5673], zoom_start=11)
# for _, row in geo_df.iterrows():
#     if row['geometry.type'] == 'Polygon':
#         folium.Polygon(
#             locations=[(lat, lon) for lon, lat in row['geometry.coordinates'][0]],
#             color='blue',
#             fill=True,
#             fill_opacity=0.2,
#             popup=row['properties.SECTEUR']
#         ).add_to(m)
# # Ajouter un marker pour chaque fait (si coordonnées disponibles)
# for _, row in faits_geo.iterrows():
#     if row['geometry.type'] == 'Polygon' and isinstance(row['geometry.coordinates'], list):
#         # print(row['geo'], row['type_matiere'], row['volume_tonnes'])

#         # Extraire tous les points du polygone
#         coords = row['geometry.coordinates'][0]
#         # Créer le polygone et calculer le centroïde
#         poly = Polygon([(lon, lat) for lon, lat in coords])
#         centroid = poly.centroid
#         folium.Marker(
#             location=[centroid.y, centroid.x],
#             popup=f"{row['type_matiere']} ({row['volume_tonnes']} t) - {row['properties.MUNICIPALITE']}",
#             icon=folium.Icon(color='green', icon='recycle', prefix='fa')
#         ).add_to(m)
# m.save("../generated/sites/collecte_secteurs.html")
# print("Carte interactive générée avec markers : collecte_secteurs.html")


# Visualisations géospatiales croisées avancées
Ce qui suit présente plusieurs exemples de visualisations géospatiales croisées :
- Carte choroplèthe (volume par secteur)
- Heatmap de points (volume ou fréquence)
- Cluster de markers (densité)
- Animation temporelle (évolution par période)
- Flux de matières (producteur → site)
- Densité par type de matière
- Points chauds (hotspots)

# Centres de tri


In [18]:
# 2bis. Afficher les centres de tri sur la carte avec leurs coordonnées GPS

import pandas as pd
import folium

# Charger les centres de tri géocodés
centres_path = '../generated/normalized/liste-centres-tri-crd.csv'
df_centres = pd.read_csv(centres_path)

# Créer la carte centrée sur le Québec
m_centres = folium.Map(location=[52, -71], zoom_start=5)

# Ajouter le geojson des secteurs (si besoin)
geojson_path = "../generated/normalized/collecte.json"
folium.GeoJson(geojson_path, name="Secteurs collecte").add_to(m_centres)

# Ajouter un marker pour chaque centre de tri avec coordonnées GPS valides
for _, row in df_centres.iterrows():
    try:
        lat = float(row['latitude'])
        lon = float(row['longitude'])
        if pd.notnull(lat) and pd.notnull(lon):
            folium.Marker(
                location=[lat, lon],
                popup=f"{row.get('Nom', 'Centre de tri')}\n{row.get('Adresse ', '')}",
                icon=folium.Icon(color='green', icon='recycle', prefix='fa')
            ).add_to(m_centres)
    except Exception:
        continue

m_centres.save('../generated/sites/centres_tri_map.html')
print("Carte interactive des centres de tri générée : centres_tri_map.html")

Carte interactive des centres de tri générée : centres_tri_map.html


In [19]:
# # Carte choroplèthe : volume total genere par secteur
# import folium
# import pandas as pd
# import json
# from branca.colormap import linear

# # Agréger le volume par secteur
# volume_par_secteur = faits.groupby('geo')['volume_tonnes'].sum().reset_index()

# # Charger le geojson
# with open(geojson_path, "r") as f:
#     geo_data = json.load(f)

# # Créer la carte
# m_choro = folium.Map(location=[45.5017, -73.5673], zoom_start=11)

# # Créer le colormap
# colormap = linear.Blues_09.scale(volume_par_secteur['volume_tonnes'].min(), volume_par_secteur['volume_tonnes'].max())
# colormap.caption = 'Volume total par secteur'
# folium.GeoJson(
#     geo_data,
#     style_function=lambda feature: {
#         'fillColor': colormap(volume_par_secteur.set_index('geo').get(feature['properties']['MUNICIPALITE'], 0)),
#         'color': 'black',
#         'weight': 1,
#         'fillOpacity': 0.6
#     },
#     tooltip=folium.GeoJsonTooltip(fields=['MUNICIPALITE'])
# ).add_to(m_choro)
# colormap.add_to(m_choro)
# m_choro.save('../generated/sites/choropleth_volume_secteur.html')
# print('Carte choroplèthe générée : choropleth_volume_secteur.html')

In [20]:
# # Heatmap de points : volume ou fréquence de collecte
# from folium.plugins import HeatMap
# import numpy as np

# # Préparer les points (centroïdes des polygones avec volume)
# points = []
# for _, row in faits_geo.iterrows():
#     if row['geometry.type'] == 'Polygon' and isinstance(row['geometry.coordinates'], list):
#         coords = row['geometry.coordinates'][0]
#         poly = Polygon([(lon, lat) for lon, lat in coords])
#         centroid = poly.centroid
#         pt = [centroid.y, centroid.x, row['volume_tonnes']]
#         # Vérifier qu'il n'y a pas de NaN
#         if not any(np.isnan(pt)):
#             points.append(pt)

# # Créer la carte heatmap
# m_heat = folium.Map(location=[45.5017, -73.5673], zoom_start=11)
# HeatMap(points, radius=15, blur=10, min_opacity=0.4, max_zoom=1).add_to(m_heat)
# m_heat.save('../generated/sites/heatmap_volume.html')
# print('Heatmap générée : heatmap_volume.html')

In [21]:
# # Cluster de markers : densité de faits
# from folium.plugins import MarkerCluster

# m_cluster = folium.Map(location=[45.5017, -73.5673], zoom_start=11)
# marker_cluster = MarkerCluster().add_to(m_cluster)

# for _, row in faits_geo.iterrows():
#     if row['geometry.type'] == 'Polygon' and isinstance(row['geometry.coordinates'], list):
#         coords = row['geometry.coordinates'][0]
#         poly = Polygon([(lon, lat) for lon, lat in coords])
#         centroid = poly.centroid
#         folium.Marker(
#             location=[centroid.y, centroid.x],
#             popup=f"{row['type_matiere']} ({row['volume_tonnes']} t) - {row['geo']}",
#             icon=folium.Icon(color='green', icon='info-sign')
#         ).add_to(marker_cluster)

# m_cluster.save('../generated/sites/cluster_markers.html')
# print('Cluster de markers généré : cluster_markers.html')

In [22]:
# # Animation temporelle : évolution du volume par période
# from folium.plugins import TimestampedGeoJson

# # Préparer les features pour chaque période
# features = []
# for _, row in faits_geo.iterrows():
#     if row['geometry.type'] == 'Polygon' and isinstance(row['geometry.coordinates'], list):
#         coords = row['geometry.coordinates'][0]
#         poly = Polygon([(lon, lat) for lon, lat in coords])
#         centroid = poly.centroid
#         feature = {
#             'type': 'Feature',
#             'geometry': {
#                 'type': 'Point',
#                 'coordinates': [centroid.x, centroid.y]
#             },
#             'properties': {
#                 'time': row['periode'],
#                 'popup': f"{row['type_matiere']} ({row['volume_tonnes']} t) - {row['geo']}"
#             }
#         }
#         features.append(feature)

# geojson_dict = {
#     'type': 'FeatureCollection',
#     'features': features
# }

# m_time = folium.Map(location=[45.5017, -73.5673], zoom_start=11)
# TimestampedGeoJson(
#     geojson_dict,
#     period='P1Y',
#     add_last_point=True,
#     auto_play=True,
#     loop=False,
#     max_speed=1,
#     loop_button=True,
#     date_options='YYYY',
#     time_slider_drag_update=True
# ).add_to(m_time)
# m_time.save('../generated/sites/animation_temporelle.html')
# print('Animation temporelle générée : animation_temporelle.html')

In [ ]:
# Flux de matières : lignes entre producteurs (territoire) et sites de valorisation (centres de tri)
import pandas as pd
import folium

# Charger les faits matières et les centres de tri
faits = pd.read_csv('../generated/faits_matieres.csv')
centres = pd.read_csv('../generated/normalized/liste-centres-tri-crd.csv')

# Nettoyer les colonnes nécessaires
for col in ['latitude', 'longitude']:
    faits[col] = pd.to_numeric(faits[col], errors='coerce')
for col in ['latitude', 'longitude']:
    centres[col] = pd.to_numeric(centres[col], errors='coerce')

# Carte centrée sur Montréal
m_flux = folium.Map(location=[45.55, -73.6], zoom_start=10)

# Ajouter un marker pour chaque territoire (producteur)
for _, row in faits.dropna(subset=['latitude', 'longitude', 'territoire']).iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Producteur: {row['territoire']}",
        icon=folium.Icon(color='blue', icon='industry', prefix='fa')
    ).add_to(m_flux)

# Ajouter un marker pour chaque centre de tri (site de valorisation)
for _, row in centres.dropna(subset=['latitude', 'longitude']).iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Site de valorisation: {row.get('Nom', 'Centre de tri')}",
        icon=folium.Icon(color='green', icon='recycle', prefix='fa')
    ).add_to(m_flux)

# Tracer les flux (lignes) entre chaque producteur et chaque site de valorisation
for _, prod in faits.dropna(subset=['latitude', 'longitude', 'territoire']).iterrows():
    for _, site in centres.dropna(subset=['latitude', 'longitude']).iterrows():
        folium.PolyLine(
            locations=[[prod['latitude'], prod['longitude']], [site['latitude'], site['longitude']]],
            color='purple', weight=1, opacity=0.3
        ).add_to(m_flux)

m_flux.save('../generated/sites/flux_matieres.html')
print('Carte des flux de matières générée : flux_matieres.html')

In [30]:
# Densité par type de matière : markers colorés par type à partir de faits_matieres.csv
import pandas as pd
import folium

# Charger les données
df = pd.read_csv('../generated/faits_matieres.csv')

# Nettoyer les colonnes nécessaires
for col in ['latitude', 'longitude', 'quantite_generee_donnees_agglo']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

type_colors = {
    'Plastique': 'red',
    'Métal': 'blue',
    'Papier': 'green',
    'Verre': 'purple',
    'Organique': 'orange',
    # Ajouter d'autres types si nécessaire
}

m_density = folium.Map(location=[45.55, -73.6], zoom_start=10)

for _, row in df.dropna(subset=['latitude', 'longitude', 'matiere', 'quantite_generee_donnees_agglo']).iterrows():
    color = type_colors.get(row['matiere'], 'gray')
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['matiere']} ({row['quantite_generee_donnees_agglo']} t)",
        icon=folium.Icon(color=color, icon='info-sign')
    ).add_to(m_density)

m_density.save('../generated/sites/densite_type_matiere.html')
print('Carte densité par type de matière générée : densite_type_matiere.html')

Carte densité par type de matière générée : densite_type_matiere.html


In [28]:
# Points chauds (hotspots) par volume à partir de faits_matieres.csv
import pandas as pd
import numpy as np
import folium

# Charger les données
df = pd.read_csv('../generated/faits_matieres.csv')

# Nettoyer les colonnes numériques
for col in ['quantite_generee_donnees_agglo', 'latitude', 'longitude']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Préparer les points avec volume (quantite_generee_donnees_agglo)
hot_points = df.dropna(subset=['latitude', 'longitude', 'quantite_generee_donnees_agglo'])
hot_points = hot_points[['latitude', 'longitude', 'quantite_generee_donnees_agglo']].values.tolist()

# Définir un seuil pour les hotspots (exemple : top 10 %)
volumes = np.array([p[2] for p in hot_points])
if len(volumes) > 0:
    seuil = np.percentile(volumes, 90)
    hotspots = [p for p in hot_points if p[2] >= seuil]

    m_hotspot = folium.Map(location=[45.55, -73.6], zoom_start=10)
    for pt in hotspots:
        folium.CircleMarker(
            location=[pt[0], pt[1]],
            radius=10,
            color='red',
            fill=True,
            fill_opacity=0.7,
            popup=f'Hotspot: {pt[2]} t'
        ).add_to(m_hotspot)

    m_hotspot.save('../generated/sites/hotspots_volume.html')
    print('Carte des hotspots générée : hotspots_volume.html')
else:
    print('Aucun point valide pour générer des hotspots.')

Carte des hotspots générée : hotspots_volume.html


In [27]:
import pandas as pd
import folium
from folium.plugins import HeatMap

# Charger les données
df = pd.read_csv('../generated/faits_matieres.csv')

# Nettoyer les colonnes numériques (au cas où il y aurait des virgules ou des valeurs manquantes)
for col in ['quantite_generee_donnees_agglo', 'quantite_collectee_donnees_agglo', 'latitude', 'longitude']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Carte centrée sur Montréal
m = folium.Map(location=[45.55, -73.6], zoom_start=10)

# Heatmap pour la quantité générée dans un FeatureGroup
fg_generee = folium.FeatureGroup(name='Quantité générée (agglo)')
heat_data_generee = df.dropna(subset=['latitude', 'longitude', 'quantite_generee_donnees_agglo'])
heat_generee = [
    [row['latitude'], row['longitude'], row['quantite_generee_donnees_agglo']]
    for _, row in heat_data_generee.iterrows()
]
HeatMap(heat_generee, min_opacity=0.4, radius=20, blur=15, max_zoom=1).add_to(fg_generee)
fg_generee.add_to(m)

# Heatmap pour la quantité collectée dans un FeatureGroup
fg_collectee = folium.FeatureGroup(name='Quantité collectée (agglo)')
heat_data_collectee = df.dropna(subset=['latitude', 'longitude', 'quantite_collectee_donnees_agglo'])
heat_collectee = [
    [row['latitude'], row['longitude'], row['quantite_collectee_donnees_agglo']]
    for _, row in heat_data_collectee.iterrows()
]
HeatMap(heat_collectee, min_opacity=0.4, radius=20, blur=15, max_zoom=1, gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'}).add_to(fg_collectee)
fg_collectee.add_to(m)

folium.LayerControl().add_to(m)
m.save('../generated/sites/heatmap_faits_matieres.html')
print("Carte HeatMap générée : heatmap_faits_matieres.html")

Carte HeatMap générée : heatmap_faits_matieres.html
